In [30]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import NoSuchElementException
import pandas as pd
import time

def setup_driver():
    """ Set up the Selenium WebDriver """
    options = webdriver.ChromeOptions()
    options.headless = True  # Run in headless mode
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=options)
    return driver
    driver = setup_driver()
    driver.get(url)
    time.sleep(5)  # Wait for JavaScript to load the data

    # Extract data using Selenium
    data = []
    rows = driver.find_elements(By.CSS_SELECTOR, "table tbody tr")
    for row in rows:
        cols = row.find_elements(By.TAG_NAME, 'td')
        row_data = [col.text for col in cols if col.text != '']  # Get text, skip empty
        if row_data:
            data.append(row_data)
    
    driver.quit()
    return pd.DataFrame(data, columns=['Skill/Potential', 'Player', 'Age', 'Team', 'ETV'])

In [31]:
def fetch_data_with_selenium(base_url, total_pages):
    driver = setup_driver()
    all_data = pd.DataFrame()

    for page in range(1, total_pages + 1):
        # Construct the URL for each page
        url = f"{base_url}/{page}"
        driver.get(url)
        time.sleep(2)  # Adjust timing based on your observation
        
        # Extract data using Selenium
        data = []
        rows = driver.find_elements(By.CSS_SELECTOR, "table tbody tr")
        for row in rows:
            try: 
                cols = row.find_elements(By.TAG_NAME, 'td')
                nationality = row.find_element(By.CSS_SELECTOR, 'td figure img').get_attribute('alt')
                row_data = [col.text for col in cols if col.text != '']
                row_data.append(nationality) 
                if row_data:
                    data.append(row_data) 
            except NoSuchElementException: 
                print("element not found") 
            
        
        # Convert page data to DataFrame and append to all_data
        page_data = pd.DataFrame(data, columns=['Skill/Potential', 'Player', 'Age', 'Team', 'ETV', 'Nationality'])
        all_data = pd.concat([all_data, page_data], ignore_index=True)

    driver.quit()
    return all_data

# URL and pages setup
base_url = 'https://www.footballtransfers.com/nl/spelers/europa'
total_pages = 50  # Total number of pages you need to scrape

# Fetch all data
data = fetch_data_with_selenium(base_url, total_pages)

In [33]:
data[['Skill', 'Potential']] = data['Skill/Potential'].str.split('\n', expand=True)

# Split 'Player' into 'Name' and 'Position'
data[['Name', 'Position']] = data['Player'].str.split('\n', expand=True)

# Drop the original 'Skill/Potential' and 'Player' columns
data.drop(columns=['Skill/Potential', 'Player'], inplace=True)

# Show the updated DataFrame
data

,Age,Team,ETV,Nationality,Skill,Potential,Name,Position
0,20,Real Madrid,€140.1M,Engeland,81.3,98.1,Jude Bellingham,"M (C), VM (RL)"
1,23,Man City,€137.4M,Noorwegen,93.8,100.0,Erling Haaland,A (CRL)
2,23,Man City,€108.5M,Engeland,89.4,98.6,Phil Foden,"AM (RL), A (C)"
3,25,PSG,€107.5M,Frankrijk,90.5,95.6,Kylian Mbappé,"A (CL), AM (L)"
4,30,Bayern,€104.5M,Engeland,85.1,86.5,Harry Kane,A (CRL)
...,...,...,...,...,...,...,...,...
1213,27,Empoli,€5.2M,Italië,63.6,65.1,Sebastiano Luperto,V (CL)
1214,32,Ajax,€5.2M,Nederland,67.8,67.8,Steven Berghuis,"M (C), AM (R)"
1215,34,Atlético,€5.1M,Spanje,69.2,69.6,César Azpilicueta,"V (CR), M (R)"
1216,20,Real Betis,€5.1M,Spanje,60.1,68.9,Chadi Riad,V (C)


In [34]:
data.to_csv('../data/eu_football_data_preprocessed.csv', index=False)